In [44]:
import nltk
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
fname_all_anecdotal_sents = "/home/shirish/BTECHSEM2/project/books/stories/new_complete_stories.txt"
fname_all_non_anecdotal_sents = "/home/shirish/BTECHSEM2/project/books/annotated_books/stories_without_anecdotes/new_all_stories.txt2"
fname_all_stories_sents = "/home/shirish/BTECHSEM2/project/books/combined_book/all_stories.txt"
fname_buffet_sents = "/home/shirish/BTECHSEM2/project/books/buffet_en.txt"
imp_cols = ["NNP","VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]

In [46]:
def tokenize(fname):
    """
    @param fname = filename
    Returns = list of sentences, where each sentence is a list of POS tagged words
    """
    f = open(fname, "r")
    taggedSents = []
    s = f.read()
    s = s.lower()
    s = unicode(s, errors="ignore")
    sentences = nltk.sent_tokenize(s)
    for i in sentences:
        taggedSents.append(nltk.pos_tag(nltk.word_tokenize(i)))
    return taggedSents

In [47]:
def featureset_df(taggedSents, value, imp_cols):
    """
    This function returns a dataframe consisting of the imp_cols and target column after
    removing the NaN values
    @param taggedSents = list of sentences, where each sentence is a list of POS tagged words
    @param value = Value given to the target column in the dataframe
    Returns = dataframe with columns as imp_cols and target with value @param value
    """
    sents1 = []
    for i in taggedSents:
        l = {}
        for j in i:
            if j[1].isalpha():
                # If Noun
                if j[1][0] == "NNP":# or j[1] == "PRP":
                    l["NNP"] = 1
                else:
                    l[j[1]] = 1
        sents1.append(l)
    df = pd.DataFrame(sents1)
    df.fillna(0, inplace=True)
    features = df[imp_cols]
    features["target"] = value
    return features

In [48]:
def tagSentences():
    """
    This function returns two tagged sentences list
    ie List of non_anecdotal , list of anecdotal sentences
    """
    taggedSents1 = tokenize(
    "/home/shirish/BTECHSEM2/project/books/stories/new_complete_stories.txt")
    taggedSents0 = tokenize(
    "/home/shirish/BTECHSEM2/project/books/annotated_books/stories_without_anecdotes/new_all_stories.txt2")
    return taggedSents0, taggedSents1
    

In [49]:
def create_feature_set_df(taggedSents0, taggedSents1):
    """
    Call this function after calling tagSentences()
    This function returns the features dataframe
    """
    features0 = featureset_df(taggedSents0, 0, imp_cols)
    if(taggedSents1 is not None):
        features1 = featureset_df(taggedSents1, 1, imp_cols)
        features = pd.concat([features1, features0])
    else:
        features = features0
    return features
    

In [50]:
def model_train(model, X_train, y_train):
    model = MultinomialNB()
    model = model.fit(X_train, y_train)
    return model

In [51]:
TS0, TS1 = tagSentences()
df = create_feature_set_df(TS0, TS1)
df = shuffle(df)
X_train, X_test, y_train, y_test = train_test_split(df[imp_cols], df["target"], test_size=0.25)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [52]:

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [53]:
df.columns


Index([u'NNP', u'VB', u'VBD', u'VBG', u'VBN', u'VBP', u'VBZ', u'target'], dtype='object')

In [58]:
model = Sequential()

In [59]:
model.add(Dense(100, input_dim=7, init='uniform', activation='sigmoid'))
model.add(Dense(50, init='uniform', activation='sigmoid'))
model.add(Dense(40, init='uniform', activation='sigmoid'))
model.add(Dense(20, init='uniform', activation='sigmoid'))
model.add(Dense(8, init='uniform', activation='sigmoid'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="sigmoid", kernel_initializer="uniform", input_dim=7)`
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="sigmoid", kernel_initializer="uniform")`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(40, activation="sigmoid", kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, activation="sigmoid", kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: Use

In [60]:
model.fit(df[imp_cols], df["target"], nb_epoch=100, batch_size=16)

Epoch 1/100
1422/1422 [==============================] - 1s 356us/step - loss: 0.6854 - acc: 0.5774
Epoch 2/100
1422/1422 [==============================] - 0s 131us/step - loss: 0.6820 - acc: 0.5774
Epoch 3/100
1422/1422 [==============================] - 0s 121us/step - loss: 0.6814 - acc: 0.5774
Epoch 4/100
1422/1422 [==============================] - 0s 126us/step - loss: 0.6814 - acc: 0.5774
Epoch 5/100
1422/1422 [==============================] - 0s 140us/step - loss: 0.6816 - acc: 0.5774
Epoch 6/100
1422/1422 [==============================] - 0s 146us/step - loss: 0.6814 - acc: 0.5774
Epoch 7/100
1422/1422 [==============================] - 0s 170us/step - loss: 0.6816 - acc: 0.5774
Epoch 8/100
1422/1422 [==============================] - 0s 157us/step - loss: 0.6814 - acc: 0.5774
Epoch 9/100
1422/1422 [==============================] - 0s 127us/step - loss: 0.6813 - acc: 0.5774
Epoch 10/100
1422/1422 [==============================] - 0s 128us/step - loss: 0.6812 - acc: 0.5774

1422/1422 [==============================] - 0s 164us/step - loss: 0.6048 - acc: 0.7046
Epoch 83/100
1422/1422 [==============================] - 0s 163us/step - loss: 0.6054 - acc: 0.7039
Epoch 84/100
1422/1422 [==============================] - 0s 155us/step - loss: 0.6049 - acc: 0.7060
Epoch 85/100
1422/1422 [==============================] - 0s 167us/step - loss: 0.6053 - acc: 0.7032
Epoch 86/100
1422/1422 [==============================] - 0s 156us/step - loss: 0.6049 - acc: 0.7060
Epoch 87/100
1422/1422 [==============================] - 0s 153us/step - loss: 0.6049 - acc: 0.7068
Epoch 88/100
1422/1422 [==============================] - 0s 158us/step - loss: 0.6048 - acc: 0.7068
Epoch 89/100
1422/1422 [==============================] - 0s 155us/step - loss: 0.6054 - acc: 0.7039
Epoch 90/100
1422/1422 [==============================] - 0s 156us/step - loss: 0.6052 - acc: 0.7060
Epoch 91/100
1422/1422 [==============================] - 0s 149us/step - loss: 0.6052 - acc: 0.7053
Epo

In [36]:
df.head(1)

,NNP,VB,VBD,VBG,VBN,VBP,VBZ,target
804,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0


In [43]:
df["target"].value_counts()

0    825
1    603
Name: target, dtype: int64